## Imports

In [1]:
import pandas as pd
import numpy as np
import json

## Load file

In [2]:
data = pd.read_csv('../data/full.csv')
print(f'Rows: {data.shape[0]}, Columns: {data.shape[1]}')

data.head()

Rows: 22343, Columns: 128


,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Patch Kosovo's iso_code

In [3]:
data.loc[data['country'] == 'Kosovo', 'iso_code'] = 'XKX'

## Select only interesting columns

In [4]:
sources = ['biofuel', 'coal', 'gas', 'hydro', 'nuclear', 'oil', 'solar', 'wind']
cols_to_keep = ['iso_code', 'country', 'year', 'population', 'per_capita_electricity']
# electricity consumption share from sources
cols_to_keep += list(map(lambda x: x + '_share_elec', sources))

data = data[cols_to_keep]
data.head()

,iso_code,country,year,population,per_capita_electricity,biofuel_share_elec,coal_share_elec,gas_share_elec,hydro_share_elec,nuclear_share_elec,oil_share_elec,solar_share_elec,wind_share_elec
0,AFG,Afghanistan,1900,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Select only europe countries

In [5]:
iso_codes = {'ALB', 'AND', 'ARM', 'AUT', 'BEL',
             'BGR', 'BIH', 'BLR', 'CHE', 'CYP',
             'CZE', 'DEU', 'DNK', 'ESP', 'EST',
             'FIN', 'FRA', 'FRO', 'GBR', 'GEO',
             'GRC', 'HRV', 'HUN', 'IMN',
             'IRL', 'ISL', 'ITA', 'LIE', 'LTU',
             'LUX', 'LVA', 'MCO', 'MDA', 'MKD',
             'MLT', 'MNE', 'NLD', 'NOR', 'POL',
             'PRT', 'ROU', 'RUS', 'SMR', 'SRB',
             'SVK', 'SVN', 'SWE', 'TUR', 'UKR',
             'VAT', 'XKX'}

data = data[data['iso_code'].isin(iso_codes)]
data.head()

,iso_code,country,year,population,per_capita_electricity,biofuel_share_elec,coal_share_elec,gas_share_elec,hydro_share_elec,nuclear_share_elec,oil_share_elec,solar_share_elec,wind_share_elec
417,ALB,Albania,1900,811069.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,ALB,Albania,1901,818784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,ALB,Albania,1902,826432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,ALB,Albania,1903,834010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,ALB,Albania,1904,841658.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# number of NaNs in population
population_nans = data['population'].isna().sum()

# number of NaNs per year in statistic data
_header = ['iso_code', 'country', 'year', 'population']
col_size = len(data.drop(columns=_header).columns)
years = set(data['year'])

# dict[year, nan_count]
data_nans: dict[int, int] = {}
for year in years:
    data_nans[year] = data[data['year'] == year].drop(columns=_header).isna().sum().sum()

for year, val in data_nans.items():
    print(f'Year: {year} --> NaNs: {val:4>}/{(data["year"] == year).sum() * col_size}')

Year: 1900 --> NaNs: 198/198
Year: 1901 --> NaNs: 198/198
Year: 1902 --> NaNs: 198/198
Year: 1903 --> NaNs: 198/198
Year: 1904 --> NaNs: 198/198
Year: 1905 --> NaNs: 198/198
Year: 1906 --> NaNs: 198/198
Year: 1907 --> NaNs: 198/198
Year: 1908 --> NaNs: 198/198
Year: 1909 --> NaNs: 198/198
Year: 1910 --> NaNs: 198/198
Year: 1911 --> NaNs: 198/198
Year: 1912 --> NaNs: 198/198
Year: 1913 --> NaNs: 198/198
Year: 1914 --> NaNs: 198/198
Year: 1915 --> NaNs: 198/198
Year: 1916 --> NaNs: 198/198
Year: 1917 --> NaNs: 198/198
Year: 1918 --> NaNs: 198/198
Year: 1919 --> NaNs: 198/198
Year: 1920 --> NaNs: 198/198
Year: 1921 --> NaNs: 198/198
Year: 1922 --> NaNs: 198/198
Year: 1923 --> NaNs: 198/198
Year: 1924 --> NaNs: 198/198
Year: 1925 --> NaNs: 198/198
Year: 1926 --> NaNs: 198/198
Year: 1927 --> NaNs: 198/198
Year: 1928 --> NaNs: 198/198
Year: 1929 --> NaNs: 198/198
Year: 1930 --> NaNs: 198/198
Year: 1931 --> NaNs: 198/198
Year: 1932 --> NaNs: 198/198
Year: 1933 --> NaNs: 198/198
Year: 1934 -->

## Filter out years with only nans

In [7]:
years_to_keep: set[int] = set()
    
for year, val in data_nans.items():
    if val != (data['year'] == year).sum() * col_size:
        years_to_keep.add(year)
        
data = data[data['year'].isin(years_to_keep)]
data.head()

,iso_code,country,year,population,per_capita_electricity,biofuel_share_elec,coal_share_elec,gas_share_elec,hydro_share_elec,nuclear_share_elec,oil_share_elec,solar_share_elec,wind_share_elec
502,ALB,Albania,1985,2969671.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,ALB,Albania,1986,3044201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,ALB,Albania,1987,3124892.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,ALB,Albania,1988,3200972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,ALB,Albania,1989,3257913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add non-existing rows for Country-year tuple (filled with NaNs)

In [8]:
min_year = data['year'].min()
max_year = data['year'].max()

countries = set(data['iso_code'])

for year in range(min_year, max_year + 1):
    for iso in countries:
        name = data[data['iso_code'] == iso]['country'].values[0]
        if data.query(f'iso_code == "{iso}" & year == {year}').empty:
            print(f'Inserting: {iso=} {year=}')
            data = pd.concat([data, pd.DataFrame({'iso_code': [iso], 'country': [name],
                        'year': [year]})], axis=0, ignore_index=True)

data.sort_values(by=['country', 'year'],inplace=True)
data.head()

Inserting: iso='MNE' year=1985
Inserting: iso='HRV' year=1985
Inserting: iso='XKX' year=1985
Inserting: iso='BIH' year=1985
Inserting: iso='SVN' year=1985
Inserting: iso='MDA' year=1985
Inserting: iso='MKD' year=1985
Inserting: iso='GEO' year=1985
Inserting: iso='SRB' year=1985
Inserting: iso='ARM' year=1985
Inserting: iso='MNE' year=1986
Inserting: iso='HRV' year=1986
Inserting: iso='XKX' year=1986
Inserting: iso='BIH' year=1986
Inserting: iso='SVN' year=1986
Inserting: iso='MDA' year=1986
Inserting: iso='MKD' year=1986
Inserting: iso='GEO' year=1986
Inserting: iso='SRB' year=1986
Inserting: iso='ARM' year=1986
Inserting: iso='MNE' year=1987
Inserting: iso='HRV' year=1987
Inserting: iso='XKX' year=1987
Inserting: iso='BIH' year=1987
Inserting: iso='SVN' year=1987
Inserting: iso='MDA' year=1987
Inserting: iso='MKD' year=1987
Inserting: iso='GEO' year=1987
Inserting: iso='SRB' year=1987
Inserting: iso='ARM' year=1987
Inserting: iso='MNE' year=1988
Inserting: iso='HRV' year=1988
Insertin

,iso_code,country,year,population,per_capita_electricity,biofuel_share_elec,coal_share_elec,gas_share_elec,hydro_share_elec,nuclear_share_elec,oil_share_elec,solar_share_elec,wind_share_elec
0,ALB,Albania,1985,2969671.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,Albania,1986,3044201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,Albania,1987,3124892.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,1988,3200972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALB,Albania,1989,3257913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Normalize share columns

In [9]:
share_cols = list(col for col in data.columns if 'share_elec' in col)
sums = data[share_cols].sum(axis=1, min_count=len(share_cols))
data[share_cols] = data[share_cols].divide(sums.fillna(100), axis='index') * 100.0

data.head()

,iso_code,country,year,population,per_capita_electricity,biofuel_share_elec,coal_share_elec,gas_share_elec,hydro_share_elec,nuclear_share_elec,oil_share_elec,solar_share_elec,wind_share_elec
0,ALB,Albania,1985,2969671.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,Albania,1986,3044201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,Albania,1987,3124892.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,1988,3200972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALB,Albania,1989,3257913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Confirm that shares add up to 100.0

In [10]:
sums = data[share_cols].sum(axis=1, min_count=len(share_cols))

print(f'Invalid rows: {(~np.isclose(sums.fillna(100), 100.0)).sum()}')

Invalid rows: 0


## Rename columns

In [11]:
name_map = {'per_capita_electricity': 'electricity_perc'}
for source in sources:
    name_map[source + '_share_elec'] = source + '_pct'

data = data.rename(name_map, axis='columns')

## Save dataset as json

In [16]:
import math

data_json = {}
iso_name_json = {}

for row in data.itertuples():
    iso_name_json[row.iso_code] = row.country
    
    iso = row.iso_code
    year = row.year
    
    if iso not in data_json:
        data_json[iso] = {}
        
    assert year not in data_json[iso]
    data_json[iso][year] = {}
    
    all_cols = ['population', 'electricity_perc'] + \
    list(map(lambda x: x + '_pct', sources))
    for col in all_cols:
        val = getattr(row, col)
        if type(val) is float and math.isnan(val):
            val = None
        data_json[iso][year][col] = val 
    
json.dump(data_json, open('../data/filtered.json', 'w'), indent=4)
json.dump(iso_name_json, open('../data/iso_name_map.json', 'w'),indent=4)

## Save dataset as csv

In [13]:
data.to_csv('../data/filtered.csv', index=False)